In [7]:
import os, json, pandas as pd, numpy as np

data_path = os.path.join(os.environ["DATAPATH"], "open", "tactile-servoing-2d-dobot")

In [8]:
# Paths and files
meta_file = os.path.join("digitac", "model_edge2d", "meta.json")

# Do split
datasets = ["train", "test"]
split = 0.75

In [11]:
# Create new dataset
for set in datasets:
    os.makedirs(os.path.join(data_path, os.path.dirname(meta_file), set), exist_ok=True) 

    # Load meta data and targets
    with open(os.path.join(data_path, meta_file), 'r') as f: 
        meta = json.load(f) 
    target_df_file = meta_file.replace("meta.json", "targets.csv")
    target_df = pd.read_csv(os.path.join(data_path, target_df_file))

    # Select data
    np.random.seed(0) # make predictable
    inds = np.random.choice([True, False], size=(meta["num_poses"]), p=[split, 1-split])
    if set=="test": inds = np.invert(inds)

    # # define new ranges
    # meta['poses_rng'][0][0] = -4; meta['poses_rng'][1][0] = 4
    # lower = target_df[['pose_1','pose_2','pose_3','pose_4','pose_5','pose_6']]>=meta['poses_rng'][0]
    # upper = target_df[['pose_1','pose_2','pose_3','pose_4','pose_5','pose_6']]<=meta['poses_rng'][1]
    # inds = np.logical_and(np.logical_and(inds, lower.all(axis=1)), upper.all(axis=1))

    # Save new dataset
    meta["meta_file"] =  os.path.join(os.path.dirname(meta_file), set, "meta.json")
    meta["target_df_file"] = os.path.join(os.path.dirname(meta_file), set, "targets.csv")
    meta["num_poses"] = len(target_df[inds])
    with open(os.path.join(data_path, meta["meta_file"]), 'w') as f: 
        json.dump(meta, f) 
    target_df[inds].to_csv(os.path.join(data_path, meta["target_df_file"]), index=False)